In [ ]:
import pandas as pd
import statsmodels.api as sm
from sklearn.preprocessing import MinMaxScaler

# ---------------------------------------------------------
# 1. SETUP & LOAD DATA
# ---------------------------------------------------------
# Load the data
df_natural = pd.read_csv("Filtration_rate_EXAMPLE_FULLFACT_natural.csv")
df_natural

In [ ]:
# ---------------------------------------------------------
# 2. MINMAX SCALER TO CONVERT NATURAL TO CODED VALUES
# ---------------------------------------------------------

# Define the Experimental Bounds (Min, Max) for all 4 factors
# Order must match columns: T, P, CoF, RPM
bounds_df = pd.DataFrame([
    [20, 1, 0.1, 1000],  # Min values
    [80, 5, 1.0, 2000]   # Max values
], columns=['T', 'P', 'CoF', 'RPM'])

scaler = MinMaxScaler(feature_range=(-1, 1))

# Fit the scaler to your defined bounds
scaler.fit(bounds_df)

# Transform your Natural Data
natural_inputs = df_natural[['T', 'P', 'CoF', 'RPM']]
coded_values = scaler.transform(natural_inputs)

# Create a DataFrame for display
df_coded = pd.DataFrame(coded_values, columns=['T', 'P', 'CoF', 'RPM'])

# Add the Response variable back to the coded DataFrame
df_coded['Filtration_rate'] = df_natural['Filtration_rate']
df_coded

In [ ]:
# ---------------------------------------------------------
# 3. FIT MODELS (REPLICATING THE EQUATIONS)
# ---------------------------------------------------------

# Define Target
y = df_coded['Filtration_rate']

# --- Model A: Natural Units ---
X_nat = df_natural[['T', 'P', 'CoF', 'RPM']]
X_nat = sm.add_constant(X_nat)  # Adds the intercept term
model_nat = sm.OLS(y, X_nat).fit()

# --- Model B: Coded Units ---
X_cod = df_coded[['T', 'P', 'CoF', 'RPM']]
X_cod = sm.add_constant(X_cod)
model_cod = sm.OLS(y, X_cod).fit()

# Print Results
print("--- 3. Regression Results Comparison ---")
print(f"{'Factor':<10} | {'Natural Coeff':<15} | {'Coded Coeff':<15}")
print("-" * 46)
params_nat = model_nat.params
params_cod = model_cod.params

for name in params_nat.index:
    print(
        f"{name:<10} | {params_nat[name]:<15.4f} | {params_cod[name]:<15.4f}")